In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
! rm ./test_db3.db

In [3]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v['description'], interval_number=v['interval_number']))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    id = Column(Integer, primary_key=True)
    borehole = Column(String(32), ForeignKey('Boreholes.id'))
    # Note that this could be a numeric ID as well
    interval_number = Column(Integer)
    components = relationship('Component',secondary='Linkintervalcomponent')
    description = Column(String(32))
    
class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [4]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2020-12-04 15:24:30,838 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-04 15:24:30,840 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:24:30,841 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-04 15:24:30,842 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:24:30,843 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-04 15:24:30,844 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:24:30,845 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-04 15:24:30,845 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:24:30,846 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-04 15:24:30,846 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:24:30,846 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-04 15:24:30,847 INFO sqlalchemy.engine.base.Engine ()
2020-12-04

In [6]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [7]:
boreholes = [Borehole(id='F11'), Borehole(id='F12'), Borehole(id='F13')] 

In [8]:
boreholes[0].intervals_values = {0: {'description':'stuff', 'interval_number':0}, 1: {'description':'junk', 'interval_number':1}}
boreholes[1].intervals_values = {2: {'description':'foo', 'interval_number':0}, 3: {'description':'bar', 'interval_number':1}}

In [9]:
c1 = Component(**{'id':0, 'description':'red fined grained limestone'})
c2 = Component(**{'id':1, 'description':'greyish sandstone'})
boreholes[0].intervals[0].components.append(c1) # TODO: specifiy the primary component
boreholes[0].intervals[0].components.append(c2)
boreholes[0].intervals[1].components.append(c2)
boreholes[1].intervals[2].components.append(c2)
boreholes[1].intervals[3].components.append(c1)

In [10]:
boreholes[0].intervals

{0: <__main__.Interval at 0x7f622490a080>,
 1: <__main__.Interval at 0x7f622490a128>}

In [11]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)

2020-12-04 15:24:30,964 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-04 15:24:30,967 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-04 15:24:30,969 INFO sqlalchemy.engine.base.Engine (('F11',), ('F12',), ('F13',))
2020-12-04 15:24:30,974 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2020-12-04 15:24:30,977 INFO sqlalchemy.engine.base.Engine ((0, 'red fined grained limestone'), (1, 'greyish sandstone'))
2020-12-04 15:24:30,980 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description) VALUES (?, ?, ?, ?)
2020-12-04 15:24:30,982 INFO sqlalchemy.engine.base.Engine ((0, 'F11', 0, 'stuff'), (1, 'F11', 1, 'junk'), (2, 'F12', 0, 'foo'), (3, 'F12', 1, 'bar'))
2020-12-04 15:24:30,984 INFO sqlalchemy.engine.base.Engine INSERT INTO "Linkintervalcomponent" (int_id, comp_id) VALUES (?, ?)
2020-12-04 15:24:30,987 INFO sqlalchemy.engine.base.Engine ((0, 0), (0,